In [1]:
from webapp import create_app
from vk_parser import *
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
import pymorphy2
%matplotlib inline

# Обработка текста.

- Обработка текста через регулярные выражения.
- Удаление всех url из текста.
- Удаление хэштегов, если они есть в тексте.
- Удаление имён пользователей.
- Удаление пунктуации и цифр.
- Приведение всего текста к нижнему регистру.

In [2]:
def preprocess_text(text):
    text = text.lower().replace('ё', 'е')
    text = text.replace('d', '')
    text = text.replace('rt', '')
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    text = re.sub('@[^\s]+', '', text)
    text = re.sub(r'[a-zA-Z]', '', text)
    text = re.sub(r'\b[a-zA-Zа-яА-Я1-9]\b', '', text)
    text = re.sub(r'\b(?:[аевзпхщчоумы][аиеоузпхщчмы]+)+\b', '', text)
    text = re.sub(r'[0-9]', '', text)    
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

# Запрос к БД на извлечение текста комментарий.

In [3]:
app = create_app()
with app.app_context():
    query = Comment.query.all()
    preprop_comm = []

    for comment in query:
        preprop_comm.append(preprocess_text(comment.comment_text))

In [6]:
bd_data = [preprocess_text(t) for t in preprop_comm]

In [7]:
print(len(bd_data))

1220


In [12]:
bd_tokenized_commens = tokenaizer(bd_data)

In [154]:
print(len(bd_tokenized_commens))

1220


In [14]:
bd_prep_comments = []
stop_words = set(stopwords.words("russian"))
for words in bd_tokenized_commens:
    for word in words:
        if word not in stop_words:
            bd_prep_comments.append(word)   

In [15]:
len(bd_prep_comments)

7215

In [16]:
morph = pymorphy2.MorphAnalyzer()
default_list = []
for word in bd_prep_comments:
    lem_word = morph.parse(str(word))[0].normal_form
    default_list.append(lem_word)

In [18]:
len(default_list)

7215

# Считаем частоту слов в нашем предобработанном датасете

In [44]:
wordfreq = {}

for token in default_list:
    if token not in wordfreq.keys():
        wordfreq[token] = 1
    else:
        wordfreq[token] += 1

In [12]:
#wordfreq

In [19]:
res = np.array(default_list)

In [20]:
unique_res = np.unique(res) 
unique_res

array(['ааа', 'ааааа', 'абаддон', ..., 'ярость', 'ясно', 'ёлочка'],
      dtype='<U19')

In [21]:
len(unique_res)

3507

In [22]:
bd_unique_words = unique_res.tolist()

# Предобработка текста.

- Используем корпус Юлии Рубцовой русских твитов для тренировки модели.
- Скачиваем корпус позитивных и негативных твитов с сайта http://study.mokoron.com/
- Добавляем дополнительную колонку для позитивных (1) и негативных твитов (0), тем самым размечая данные.

In [3]:
# Считываем данные
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('training_data/positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('training_data/negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

# Формируем сбалансированный датасет
sample_size = min(data_positive.shape[0], data_negative.shape[0])

raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

In [4]:
data = [preprocess_text(t) for t in raw_data]

In [5]:
data

['хоть школота но поверь нас то же самое общество профилирующий предмет типа',
 'да все таки он немного похож на него но мой мальчик все равно лучше',
 'ну ты идиотка испугалась тебя',
 'кто то углу сидит погибает от голода порции взяли хотя уже так жрать не хотим',
 'вот что значит страшилка но блин посмотрев все части тебя создастся ощущение что авторы курили что то',
 'ну любишь или нет не знаю кто ты бля',
 'ну это конечно же чтобы нее было много друзей ведь она такая',
 'тебя есть ухажер нет уши не кто не жрет',
 'поприветствуем моего нового читателя',
 'теперь меня есть частичка сиднея',
 'люблю остальное так влюбляюсь привязываюсь',
 'как то забыла что вчера приехал из деревни наш котэня испугалась когда он утром пришел ко мне общаться доброе',
 'так было задумано да',
 'дааааа ты ты ты только ты',
 'нет ведь все равно обзоры урезаете просто одна версия сильно порезанная другая полная только без мусора',
 'смотри там случайно аспирантуру не попади наука никого до добра не доводи

In [7]:
print(len(labels), len(raw_data), len(data))

223846 223846 223846


- Скачиваем набор русских стоп-слов для предобработки наших твитов.

In [8]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kamikaze666\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kamikaze666\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

- Разбиваем предобработанные слова на токены.
- Удаляем стоп слова.
- Производим лемматизацию или стемминг текста.
- Удаляем короткие слова длинной меньше 2 символов.

In [10]:
#%timeit tokenized_words = [word_tokenize(sent) for sent in data]

In [11]:
def tokenaizer(data):
    tokens= [word_tokenize(sent) for sent in data]
    return tokens

In [12]:
tokenized_words = tokenaizer(data)

In [38]:
print(len(tokenized_words))

223846


# Стемминг слов

In [17]:
snowball = SnowballStemmer(language="russian")
def stemmer(word, stem_init):
    return stem_init.stem(str(word))

In [18]:
stop_words = set(stopwords.words("russian"))
def delete_stop_words_with_stemming(words, stop_words):    
    default_list = []
    for word in words:
        if word not in stop_words:
            stemmed_word = stemmer(word, snowball)
            default_list.append(stemmed_word)

    return list(default_list)

In [8]:
def to_data_frame(data, labels):
    default_list = []
    
    
    for string in data:
        new_string = ' '.join(string)
        default_list.append(new_string)
    
    
    default_list = pd.DataFrame(list(zip(default_list, labels)), columns =['comments', 'sentiment'])
    return default_list

In [43]:
stemmed_words = []
counter = 0
for part_of_list in tokenized_words:
    stemmed_words.append(delete_stop_words_with_stemming(part_of_list, stop_words))
    counter += 1
    if counter%1000 == 0:
        print("Processed {} of {}".format(counter, len(part_of_list)))

Processed 1000 of 7
Processed 2000 of 6
Processed 3000 of 15
Processed 4000 of 9
Processed 5000 of 14
Processed 6000 of 14
Processed 7000 of 7
Processed 8000 of 16
Processed 9000 of 19
Processed 10000 of 11
Processed 11000 of 7
Processed 12000 of 6
Processed 13000 of 9
Processed 14000 of 8
Processed 15000 of 13
Processed 16000 of 8
Processed 17000 of 6
Processed 18000 of 11
Processed 19000 of 9
Processed 20000 of 10
Processed 21000 of 11
Processed 22000 of 2
Processed 23000 of 7
Processed 24000 of 12
Processed 25000 of 7
Processed 26000 of 11
Processed 27000 of 10
Processed 28000 of 5
Processed 29000 of 13
Processed 30000 of 11
Processed 31000 of 16
Processed 32000 of 3
Processed 33000 of 5
Processed 34000 of 7
Processed 35000 of 17
Processed 36000 of 14
Processed 37000 of 6
Processed 38000 of 9
Processed 39000 of 16
Processed 40000 of 5
Processed 41000 of 6
Processed 42000 of 7
Processed 43000 of 6
Processed 44000 of 7
Processed 45000 of 9
Processed 46000 of 13
Processed 47000 of 9
Pr

In [ ]:
print(len(stemmed_words))

In [ ]:
stemmed_words = to_data_frame(stemmed_words, labels)

# Лемматизация слов.

"Экземпляры класса MorphAnalyzer обычно занимают порядка 15Мб оперативной памяти (т.к. загружают в память словари, данные для предсказателя и т.д.); старайтесь организовать свой код так, чтобы создавать экземпляр MorphAnalyzer заранее и работать с этим единственным экземпляром в дальнейшем."

https://pymorphy2.readthedocs.io/en/latest/user/guide.html

In [13]:
morph = pymorphy2.MorphAnalyzer()
stop_words = set(stopwords.words("russian"))
def delete_stop_words_with_lemmatizing(words, morph, stop_words):  
    default_list = []
    for word in words:
        if word not in stop_words:
            lem_word = morph.parse(str(word))[0].normal_form
            default_list.append(lem_word)
    return list(default_list)

In [14]:
lemmatized_words = []
counter = 0
for part_of_list in tokenized_words:
    lemmatized_words.append(delete_stop_words_with_lemmatizing(part_of_list, morph, stop_words))   
    counter += 1
    if counter%1000 == 0:
        print("Processed {} of {}".format(counter, len(part_of_list)))

Processed 1000 of 7
Processed 2000 of 5
Processed 3000 of 15
Processed 4000 of 8
Processed 5000 of 13
Processed 6000 of 13
Processed 7000 of 7
Processed 8000 of 16
Processed 9000 of 18
Processed 10000 of 11
Processed 11000 of 7
Processed 12000 of 5
Processed 13000 of 9
Processed 14000 of 8
Processed 15000 of 13
Processed 16000 of 8
Processed 17000 of 6
Processed 18000 of 9
Processed 19000 of 9
Processed 20000 of 10
Processed 21000 of 10
Processed 22000 of 2
Processed 23000 of 6
Processed 24000 of 11
Processed 25000 of 7
Processed 26000 of 11
Processed 27000 of 10
Processed 28000 of 4
Processed 29000 of 13
Processed 30000 of 9
Processed 31000 of 14
Processed 32000 of 3
Processed 33000 of 4
Processed 34000 of 6
Processed 35000 of 16
Processed 36000 of 14
Processed 37000 of 6
Processed 38000 of 9
Processed 39000 of 15
Processed 40000 of 5
Processed 41000 of 6
Processed 42000 of 7
Processed 43000 of 6
Processed 44000 of 5
Processed 45000 of 9
Processed 46000 of 12
Processed 47000 of 9
Proc

In [15]:
len(lemmatized_words)

223846

In [16]:
lemmatized_words = to_data_frame(lemmatized_words, labels)

In [18]:
lemmatized_words = lemmatized_words.dropna().reset_index(drop=True)
lemmatized_words.isnull().sum()

comments     0
sentiment    0
dtype: int64

# Подготовка лемматизированных данных для обучения, валидации и тестирования.

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(lemmatized_words['comments'],lemmatized_words['sentiment'],test_size = 0.90, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size = 0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test,test_size = 0.5, random_state=42)

print("Data distribution:\n- Train: {} \n- Validation: {} \n- Test: {}".format(len(y_train),len(y_val),len(y_test)))

Data distribution:
- Train: 11192 
- Validation: 5596 
- Test: 5596


In [33]:
X_train

172855    завтра ужасно пойти школа пойти ладный выспать...
23908            выражение услышать паршивый овца бочка мёд
172208                        задержка вылет час нафиг жить
162818                                бля вообще лень учить
33933                            помнить лето рука валанчик
                                ...                        
13777                                        дан цвет волос
144188                              такой чувство бить ночь
440                   завтра школа гулять ждать наш встреча
41911                       поставить новый клавиатура свой
156488                                   алина пойти пофига
Name: comments, Length: 11192, dtype: object

In [34]:
y_train

172855    0
23908     1
172208    0
162818    0
33933     1
         ..
13777     1
144188    0
440       1
41911     1
156488    0
Name: sentiment, Length: 11192, dtype: int64

# Word embedding (Векторное представление слов)

- Сделаем векторизацию текста 4-мя способами BoW, TF-IDF, Word2Vec, doc2vec.
- На этих данных обучим модели и сравним результаты.

# Bag Of Words (BoW).

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

In [62]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(lemmatized_words['comments'])

In [68]:
print(vectorizer.get_feature_names()[:100])

['аа', 'ааа', 'аааа', 'ааааа', 'аааааа', 'ааааааа', 'аааааааа', 'ааааааааа', 'аааааааааа', 'ааааааааааа', 'аааааааааааа', 'ааааааааааааа', 'аааааааааааааа', 'ааааааааааааааа', 'аааааааааааааааа', 'ааааааааааааааааа', 'аааааааааааааааааа', 'ааааааааааааааааааа', 'аааааааааааааааааааа', 'ааааааааааааааааааааа', 'аааааааааааааааааааааа', 'ааааааааааааааааааааааа', 'аааааааааааааааааааааааа', 'ааааааааааааааааааааааааа', 'аааааааааааааааааааааааааа', 'ааааааааааааааааааааааааааа', 'ааааааааааааааааааааааааааааааа', 'аааааааааааааааааааааааааааааааа', 'ааааааааааааааааааааааааааааааааа', 'аааааааааааааааааааааааааааааааааа', 'аааааааааааааааааааааааааааааааааааа', 'аааааааааааааааааааааааааааааааааааааааа', 'ааааааааааааааааааааааааааааааааааааааааааааааааааааа', 'аааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааа', 'ааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааа', 'аааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааааа', 'ааааааааааааааааааа

In [39]:
print(X.shape)

(223846, 94121)


# Bag Of Word with n-grams.

In [64]:
vectorizer2 = CountVectorizer(ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(lemmatized_words['comments'])

In [65]:
print(vectorizer2.get_feature_names()[:100])

['аа аа', 'аа ааа', 'аа айн', 'аа акаарыбын', 'аа ап', 'аа арай', 'аа ардчилал', 'аа ахи', 'аа бантик', 'аа бедный', 'аа бифри', 'аа бла', 'аа блин', 'аа бля', 'аа блянулааан', 'аа бог', 'аа болеть', 'аа болзоон', 'аа брат', 'аа быть', 'аа верить', 'аа вон', 'аа вообще', 'аа воот', 'аа вспомнить', 'аа вчера', 'аа вще', 'аа выспаться', 'аа выходить', 'аа говорить', 'аа год', 'аа горло', 'аа господь', 'аа гэж', 'аа даа', 'аа давать', 'аа делать', 'аа день', 'аа дианыч', 'аа долго', 'аа думать', 'аа жарко', 'аа ждать', 'аа заболеть', 'аа забыть', 'аа завтра', 'аа заебалиия', 'аа закончиться', 'аа замёрзлый', 'аа запутаться', 'аа зараз', 'аа заставить', 'аа зато', 'аа збс', 'аа значит', 'аа извинить', 'аа ита', 'аа каждый', 'аа казаться', 'аа капец', 'аа каток', 'аа киев', 'аа класс', 'аа ключ', 'аа код', 'аа красота', 'аа крилла', 'аа ладный', 'аа лан', 'аа любить', 'аа маленький', 'аа мочь', 'аа наверно', 'аа нада', 'аа наивный', 'аа народ', 'аа начать', 'аа начаться', 'аа недавно', 'аа 

In [66]:
print(X2.shape)

(223846, 781179)


- Вывод: 
  - Потеря порядка слов;
  - Полученные вектора, не являются информативными и занимают очень много места в памяти;

# TF-IDF.

(TF — term frequency, IDF — inverse document frequency) — статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса.

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize(data,tfidf_vect_fit):
    X_tfidf = tfidf_vect_fit.transform(data)
    words = tfidf_vect_fit.get_feature_names()
    X_tfidf_df = pd.DataFrame(X_tfidf.toarray())
    X_tfidf_df.columns = words
    return(X_tfidf_df)

In [36]:
tfidf_vect = TfidfVectorizer()
tfidf_vect_fit = tfidf_vect.fit(X_train)
X_train = vectorize(X_train,tfidf_vect_fit)

In [37]:
X_train.shape

(11192, 15697)

In [38]:
y_train.shape

(11192,)

# Random Forest Classifier.

- Мы исследуем первые результаты классификатора Random Forest без какой-либо настройки гиперпараметров с использованием 5-кратной проверки.

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier()
scores = cross_val_score(clf, X_train, y_train.values.ravel(), cv=5)

In [40]:
print(scores)
scores.mean()

[0.6342117  0.63287182 0.6510277  0.63092046 0.62957998]


0.6357223339124729

# Decision Tree Classifier.

In [42]:
from sklearn import tree

In [47]:
DTC_clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)

In [48]:
DTC_clf.fit(X_train, y_train.values.ravel())

DecisionTreeClassifier(criterion='entropy', max_depth=4)

In [49]:
DTC_clf.score(X_train, y_train.values.ravel())

0.5243030736240172

In [51]:
#DTC_clf.score(X_test, y_test.values.ravel()) #X_test нужно векторизовать